## REINFORCE with PyTorch

First of all, let's the update step of the REINFORCE algorithm:

$$\theta \leftarrow \theta + \alpha \gamma^t G_t \nabla_\theta \log\pi(A_t|S_t, \theta).$$

Here, we assume that we ran a whole episode (i.e., we follows some policy parametrized by $\theta$ until the end, and now we want to update $\theta$. The return $G_t$ is the total reward we got starting at time-step t. The $\gamma$ is the discount rate, which we will assume is equal to $1$ from now on. The probability of taking action $A_t$ (which we took at time $t$) using the current policy is $\pi(A_t|S_t, \theta)$.



### The Iterated Prisoners' Dilemma

Here is the task that we want to solve.

Two prisoners are considering whether to give evidence to the police. The prisoners cannot talk to each other. The prisoners can cooperate with each other and not give evidence to the police: in that case the reward to each of the prisoners is -1. They can both give evidence to the police, in which case the reward for both is -2 (we say that both prisoners "defected"). If prisoner A gives evidence to the police but prisoner B does not, prisoner A gets a reward of 0, and prisoner B gets a reward of -3.

If this scenario occurs the once, the prisoners can just guess, but if the scneario repeats multiple times, the prisoners can try to see what happened last time, and act accordingly.

### Tit-for-tat

Robert Axelrod at MIT showed in the 80s that a "tit-for-tat" strategy is very effective: prisoner A should co-operate if prisoner B co-operated last time, and defect optherwise.

Let's encode the game first:

In [1]:
import torch
import numpy as np

from torch.autograd import Variable


def agent_ttt(act_agent_prev, act_user_prev):
    return act_user_prev
    
def agent_evil(act_agent_prev, act_user_prev):
    return 0
    
def agent_nice(act_agent_prev, act_user_prev):
    return 1
        
def reward(act_agent, act_user):
    '''Return the reward for the user
       act_agent -- the action of the agent (0 is defect, 1 is cooperate)
       act_user --  the action of the user (0 is defect, 1 is cooperate)
    '''
    # 0: defect
    # 1: cooperate
    # matrix:
    #   
    
    #   agent             user
    #
    #               defect    cooperate
    #  defect       -2, -2         0, -3
    #  cooperate    -3,  0        -1, -1            
    #
    reward_matrix = {(0, 0): -2.0,
                     (1, 1): -1.0,
                     (0, 1): -3.0,
                     (1, 0):  0.0}
    
    return reward_matrix[(act_agent, act_user)]
    
    
    
    

map_states =  {(0, 0): 0,
               (1, 1): 1,
               (0, 1): 2,
               (1, 0): 3}


### Learning with REINFORCE

Now, let's set up the logits for cooperating in each of the four possible scenarios. We use the logits to avoid doing gradient descent on variables whose values are outside of $0..1$.

In [2]:
import torch
import numpy as np

from torch.autograd import Variable

q = Variable(torch.randn(4), requires_grad=True)   # P(cooperate) for
                                                   # (0, 0), (1, 1), (0, 1), (1, 0)
                                                   # in the previous game


We are now ready to set up REINFORCE

In [3]:
N = 1000
n_iter = 100


learning_rate = 0.0001

for i in range(N):
    act_agent = np.random.randint(2)
    act_user = np.random.randint(2)
    G = []
    logPa = [] 
    a = []
    
    agent_fn = [agent_ttt, agent_evil, agent_nice][np.random.randint(3)]
    
    
    for iter in range(n_iter):
        
        prob_cur = torch.sigmoid(q[map_states[(act_agent, act_user)]])
        
        act_agent, act_user = agent_fn(act_agent, act_user), np.random.binomial(1, prob_cur.data.numpy()[0])
        
        g_cur = reward(act_agent, act_user)
        
        a.append(act_user)
        G.append(g_cur)
        if act_user == 0:
            logPa.append(torch.log(1-prob_cur))
        else:
            logPa.append(torch.log(prob_cur))
    
    returns = np.cumsum(G[::-1])[::-1]
    
    for iter in range(n_iter):
        weighted_pa = logPa[iter] * returns[iter]
        weighted_pa.backward()
        
    
    #print(q.grad.data.numpy())
    
    q.data = q.data + learning_rate * q.grad.data
    q.grad.data.zero_()
    
    print(q.data.numpy())
    
    print("Reward", returns[0])
        
    
    
        
        

[-0.45844042  0.81211728  0.06531569 -0.34776628]
Reward -145.0
[-0.41728044  0.80939823  0.06205814 -0.35451803]
Reward -152.0
[-0.35751712  0.8020674   0.03924663 -0.35451803]
Reward -238.0
[-0.35751712  0.80139768  0.03924663 -0.36373401]
Reward -58.0
[-0.32069746  0.81225938  0.08231825 -0.35226315]
Reward -160.0
[-0.29687259  0.81225938  0.02851024 -0.35226315]
Reward -248.0
[-0.2503714   0.81225938  0.0399952  -0.34256157]
Reward -235.0
[-0.27821168  0.81613123  0.0533538  -0.32511109]
Reward -142.0
[-0.27821168  0.82057798  0.05082314 -0.31564316]
Reward -52.0
[-0.32734451  0.83160269  0.03800436 -0.304699  ]
Reward -143.0
[-0.32734451  0.81764704  0.03800436 -0.31406808]
Reward -68.0
[-0.33298314  0.79767108  0.02101446 -0.30158296]
Reward -134.0
[-0.31650686  0.80085677  0.02822366 -0.27629787]
Reward -153.0
[-0.31531721  0.83779562  0.01234011 -0.30737823]
Reward -144.0
[-0.31547597  0.83779562  0.03738564 -0.2968691 ]
Reward -248.0
[-0.39821318  0.83779562 -0.02136282 -0.296